# Slik kan du bruke Felles datakatalog med Python
Etter litt privat "Summer of Code" med mobiltelefonen, et eksternt tastatur og e-boka ["Head First Python"](https://books.google.no/books?id=WYqNDQAAQBAJ&lpg=PP1&hl=no&pg=PP1#v=onepage&q&f=false) har jeg lyst å snikskryte litt ved å vise fire ting jeg synes er veldig kule:

1) Programmeringsspråket Python,
2) Python Notebook,
3) [Felles datakatalog](https://fellesdatakatalog.brreg.no)
4) API-et for åpne data fra Enhetsregisteret

## 1. Programmeringsspråket Python
Som alle sikkert har fått med seg er det mye snakk om at kidsa må kode. Kodetimen bidrar til at mange barn får en viss erfaring med at det er mulig å styre "magien" i datamaskinene. Istedenfor å kun spille spill, kan de også __lage__ spill. Så hvis dette er viktig og riktig for kidsa, så mener jeg det også er riktig for vi som kaller oss digitaliseringsbyråkrater (eller noe tilsvarende sexy). Men hvordan kommer man igang? En mulighet er å starte med Python. Her. Og. Nå.

Klikk i cellen nedenfor og skriv inn navnet ditt. Og trykk på "play"-knappen øverst eller hold skift-knappen inne mens du trykker enter:

In [ ]:
navn = "[Skriv navnet ditt her]"
print(navn)

Hvis du ikke fikk en feilmelding så har variabelen "navn" nå navnet ditt som sin verdi. Da kan vi telle hvor langt navnet er, skrive det baklengs, finne vokaler osv:

In [ ]:
len(navn) # teller antall tegn

In [ ]:
for b in navn[-1::-1]: # starter bakerst og skriver ut bokstavene
    print(b, end="")

In [ ]:
set(navn) & set('aeiouyæøå') # finner vokalene i navnet ditt ved å se på snittet mellom de to bokstavmengdene

In [ ]:
navn.isupper() # Har du skrevet navnet ditt med bare store bokstaver?

## 2. Python Notebook
(eller Jupyter som jeg tror det heter nå)
I eksemplene over har du brukt programmeringsspråket Python. Tradisjonelt krever programmering at det installeres verktøy for å redigere og kjøre programkoden på PC-en. Det trenger ikke å være komplisert, og er vel verdt innsatsen hvis man har lyst til å lære programmering. Men jeg synes det er superpraktisk at det også er mulig å komme igang i en slik "notebook" som du bruker nå. 

Du er ikke begrenset av den koden jeg har skrevet inn, men kan legge mer kode og prøve deg frem. Ved å bruke kommandoen dir(navn) får du opp en liste ting variabelen "navn" kan gjøre. Deretter kan du be om hjelp for de ulike funksjonene. F.eks. å endre fra små til store bokstaver, og motsatt. Prøv f.eks. swapcase. (Husk at du må bruke punktum mellom navn og funksjonsnavnet på samme måte som navn.isupper() i det siste eksempelet over.)

In [ ]:
dir(navn)

In [ ]:
help(navn.swapcase)

In [ ]:
# Prøv deg frem selv her (eller et annet sted), f.eks. navn.swapcase()

# 3. Felles datakatalog
Innrømmelse: Jeg jobber i Brønnøysundregistrene med Felles datakatalog, så jeg er selvsagt ikke helt objektiv ...

Felles datakatalog inneholder blant annet en API-katalog. Og Felles datakatalog har (naturligvis!) også et API. Og API-er er laget for at maskiner skal kunne samhandle. Så la oss bruke Python til å snakke med Felles datakatalog som maskiner gjør det, fremfor å gå med nettleseren til datakatalogen bruke det mennesketilpassede grensesnittet.

NB! Husk at du må kjøre cellene nedenfor en etter en, for det som skjer i en celle er avhengig av det som har skjedd i cellene før.

In [ ]:
import requests # requests er et superpopulært tillegg til Python som gjør det enkelt å sende forespørsler til andre servere

In [ ]:
headers = { 'Accept': 'application/json'} # vi vil snakke slik moderne maskiner snakker sammen, med JSON
base_url = 'https://fellesdatakatalog.brreg.no/api/apis' # dette er adressen til API-et til API-katalogen ...

Da har vi en adresse å sende en forespørsel til (base_url) og en spesifikasjon av hvordan vi vil ha svaret på forespørselen. Vi må i tillegg lagre svaret slik at vi kan bruke det videre.

In [ ]:
r = requests.get(base_url, headers = headers) # Svaret på forespørselen lagres i variabelen r

In [ ]:
r.status_code # statuskoden bør være 200 -- som er koden for suksess i http

In [ ]:
r.headers['content-type'] # vi kan sjekke om vi fikk JSON

In [ ]:
r.json() # og se på det vi fikk (NB! her kommer det (forhåpentligvis ...) mye ut)

Over har vi fått listet innholdet på den første siden av API-katalogen. La oss søke etter et konkret API, nemlig API-et til åpne data fra Enhetsregisteret. Det gjør vi ved å legge søkeordet i variabelen s, og lage en ny adresse som inneholder søkeordet som et spørreparameter:

In [ ]:
s = 'enhetsregisteret' 
r = requests.get(base_url + '?q=' + s, headers = { 'Accept': 'application/json'})
r.status_code

Hvis du fikk '200' over, var forespørselen vellykket, og vi har fått data lagret i r. Vi ser på resultatet:

In [ ]:
r.json()

Enhetsregisteret er omtalt i to API-beskrivelser. Det første treffet er det vi ønsker oss og for å referere til den beskrivelsen må vi se på den delen av svaret som handler om "hits", og velge den første av de to. Og i programmeringsspråk er den første nummer 0 (slik 1. etasje er "nullte etasje" i andre land). I Python blir det sånn:

In [ ]:
r.json()['hits'][0]

Over har vi deler av informasjonen som vises i Felles datakatalog om API-et "Åpne data fra Enhetsregisteret". Du kan se det i nettleseren her: https://fellesdatakatalog.brreg.no/apis/3b88f2d7-8531-4e5c-a0a0-e22ff96221df
Hvis vi vil snakke med API-et for Åpne data fra Enhetsregisteret "maskin til maskin", så trenger vi å vite flere tekniske detaljer, som adressen til serveren og hvilke kall man kan gjøre til API-et og formen på disse. Den tekniske spesifikasjonen av API-et får man ved å legge til id-en for det API-et man er interessert i, samt '/spec' til adressen til API-katalogen.

In [ ]:
r.json()['hits'][0]['id'] # hente ut id-en til API-beskrivelsen over

Tidligere lagret vi url-en til API-katalogen i base_url, så nå kan vi sette bitene sammen:

In [ ]:
spec_url = base_url + '/' + r.json()['hits'][0]['id'] + '/spec'
print(spec_url)

Da kan vi hente spesifikasjonsfilen:

In [ ]:
spec = requests.get(spec_url, headers = { 'Accept': 'application/json'})

In [ ]:
spec.json()

## 4. Åpne data fra Enhetsregisteret
Enhetsregisteret er den autoritative kilden til opplysninger om enheter (bedrifter, statlige og kommunale virksomheter osv) i Norge. I motsetning til i mange andre land er mange av disse opplysningene gjort tilgjengelige som åpne data, og gjennom et moderne API. Så nå kan vi bruke informasjon fra spesifikasjonen vi lastet ned over til å gjøre en forespørsel om f.eks. Equinor ASA, som har organisasjonsnummer 923609016, og se hvilken næringskode de har -- det vil si hva som er deres primære aktivitet.

Fra spesifikasjonen vi lastet ned over finner jeg adressen til serveren for API-et og hvordan jeg skal utforme en forespørsel:

In [ ]:
enhet_url = spec.json()['servers'][0]['url'] + '/enheter/923609016'
print(enhet_url)

In [ ]:
enhet = requests.get(enhet_url, headers=headers) # sender forespørsel og lagrer resultatet i variabelen 'enhet'

In [ ]:
enhet.status_code # suksess?

In [ ]:
enhet.json() # Hva har vi fått?

In [ ]:
enhet.json()['naeringskode1'] # Hva driver Equinor med?

Felles datakatalog kan også gi en liste over servere (endepunkter) som implementerer det samme API-et. Dette er f.eks. tilfelle der alle bankene har blitt enige om et standard API for å tilgjengeliggjøre informasjon om saldo.

In [215]:
saldo_endepunkt = requests.get('https://demo.fellesdatakatalog.brreg.no/api/apis/endpoints?serviceType=Kontoopplysninger', headers=headers)
print(saldo_endepunkt.json())

{'total': 0, 'endpoints': []}
